In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ls /content/drive/My\ Drive/Colab

cleanTrainCap  dataloader.ipynb			images
cleanTrainID   data_prep.ipynb			invalidAllUrls
codec.ipynb    data_prep.py			main.ipynb
codec.py       eee443_project_dataset_train.h5


In [0]:
import torch
import os
from torch.utils.data import DataLoader
from torch import nn
from torchvision import transforms
from PIL import Image
import pandas as pd
#from codec import EncoderCNN, DecoderRNN
#from data_prep import ImageDataset

In [0]:
import torch
import torchvision
from torch import nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class EncoderCNN(nn.Module):
    def __init__(self, embed_size=1024):
        super(EncoderCNN, self).__init__()

        # get the pretrained densenet model
        self.densenet = torchvision.models.densenet121(pretrained=True)

        # replace the classifier with a fully connected embedding layer
        self.densenet.classifier = nn.Linear(in_features=1024, out_features=1024)

        # add another fully connected layer
        self.fulCon = nn.Linear(in_features=1024, out_features=embed_size)

        # dropout layer
        self.dropout = nn.Dropout(p=0.5)

        # activation layers
        self.prelu = nn.PReLU()

    def forward(self, images):

        # get the embeddings from the densenet
        densenet_outputs = self.dropout(self.prelu(self.densenet(images)))

        # pass through the fully connected
        embeddings = self.fulCon(densenet_outputs)

        return embeddings
    def fine_tune(self):
#        for p in self.densenet.parameters():
#            p.requires_grad = False
        for c in list(self.densenet.children())[0][:6]:
            for p in c.parameters():
                p.requires_grad = False

class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers=1):
        super(DecoderRNN, self).__init__()

        # define the properties
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size

        # embedding layer
        self.embed = nn.Embedding(num_embeddings=self.vocab_size, embedding_dim=self.embed_size)

        # lstm cell
        self.lstm_cell = nn.LSTMCell(input_size=embed_size, hidden_size=hidden_size)

        # output fully connected layer
        self.fc_out = nn.Linear(in_features=self.hidden_size, out_features=self.vocab_size)

        # activations
        self.softmax = nn.Softmax(dim=1)

    def forward(self, features, captions):
        # batch size
        batch_size = features.size(0)

        # init the hidden and cell states to zeros
        hidden_state = torch.zeros((batch_size, self.hidden_size)).to(device)
        cell_state = torch.zeros((batch_size, self.hidden_size)).to(device)

        # define the output tensor placeholder
        outputs = torch.empty((batch_size, captions.size(1), self.vocab_size)).to(device)

        # embed the captions
        captions_embed = self.embed(captions)

        # pass the caption word by word
        for t in range(captions.size(1)):

            # for the first time step the input is the feature vector
            if t == 0:
                hidden_state, cell_state = self.lstm_cell(features.to(device), (hidden_state, cell_state))

            # for the 2nd+ time step, using teacher forcer
            else:
                hidden_state, cell_state = self.lstm_cell(captions_embed[:, t, :], (hidden_state, cell_state))

            # output of the attention mechanism
            out = self.fc_out(hidden_state)

            # build the output tensor
            outputs[:, t, :] = out

        return outputs

In [0]:
import pickle
import numpy as np
import torch
from torch.utils.data import Dataset
from PIL import Image

class ImageDataset(Dataset):
    """MIT image dataset."""

    def __init__(self, root_dir, data, transform = None):
        self.root_dir  = root_dir
        self.dataPtr   = data
        self.transform = transform
        with open(self.root_dir + "cleanTrainCap", "rb") as f:
            self.trainCap = pickle.load(f)
        with open(self.root_dir + "cleanTrainID", "rb") as f:
            self.trainID = pickle.load(f) - 1

    def __len__(self):
        return len(self.dataPtr)

    def __getitem__(self, idx):
        img_name = self.root_dir + "images/" + self.dataPtr[idx]
        image = Image.open(img_name)
        if image.mode == 'L':
            image = image.convert('RGB')
        elif image.mode == 'CMYK':
            image = image.convert('RGB')
            
#        if image.size[0] == 1:
#            image = image.repeat(3,1,1)
#
#        elif  image.size[0] == 4:
#            image = image[0].repeat(3,1,1)
            
        if self.transform:
            image = self.transform(image)

        imgNum = int(self.dataPtr[idx][3:-4])
        captions = self.trainCap[self.trainID == imgNum]
        rand    = np.random.randint(captions.shape[0])
        caption = captions[rand]

        sample = {'image': image, 'caption': torch.from_numpy(np.array(caption))}
        return sample

In [7]:
torch.cuda.is_available()


True

In [9]:
root = "/content/drive/My Drive/Colab/"
totalList = (os.listdir(root+"images/"))
#print(len(totalList))

OSError: ignored

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Load data
root = "/content/drive/My Drive/Colab/"
#totalList = sorted(os.listdir(root+"images/"), key=len)
totalList = os.listdir(root+"images/")
lens = [int(len(totalList)*0.5)+1, int(len(totalList)*0.4), int(len(totalList)*0.1)+1]
train, test, val = torch.utils.data.dataset.random_split(totalList, lens)

dataset_tr   = ImageDataset(root_dir = root, data = train, transform = transforms.Compose([transforms.RandomHorizontalFlip(), transforms.Resize((224,224),interpolation=Image.NEAREST),
                                                                                           transforms.ToTensor(),transforms.Normalize(mean = [0.489, 0.456, 0.406], std = [0.229, 0.224,0.225])]))
dataset_test = ImageDataset(root_dir = root, data = test, transform = transforms.Compose([transforms.Resize((224,224),interpolation=Image.NEAREST), transforms.ToTensor()]))
dataset_val  = ImageDataset(root_dir = root, data = val, transform = transforms.Compose([transforms.Resize((224,224),interpolation=Image.NEAREST),
                                                                                         transforms.ToTensor(), transforms.Normalize(mean = [0.489, 0.456, 0.406], std = [0.229, 0.224,0.225])]))
dataloader_train = DataLoader(dataset_tr, batch_size=16, shuffle=True)
#dataloader_train2 = DataLoader(dataset_tr, batch_size=32, shuffle=True)
#dataloader_test  = DataLoader(dataset_test, batch_size=1, shuffle=False)
dataloader_val   = DataLoader(dataset_val, batch_size=100, shuffle=True)

# Initializations for training
losses = list()
val_losses = list()
decoder = DecoderRNN(embed_size=100, hidden_size=128, vocab_size=1004).to(device)
encoder = EncoderCNN(100).to(device)
encoder.fine_tune()
criterion = nn.CrossEntropyLoss().to(device)

vocab_size = 1004
encoder_lr = 0.008
encoder_optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, encoder.parameters()),
                                             lr=encoder_lr)
decoder_lr = 0.008
decoder_optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, decoder.parameters()),
                                             lr=decoder_lr)

wordC = pd.read_hdf(root + "eee443_project_dataset_train.h5", 'word_code')
wordC = wordC.to_dict('split')
wordDict = dict(zip(wordC['data'][0], wordC['columns']))
# Training starts
for epoch in range(1, 5):
    print("Epoch:",epoch)
    for i, data in enumerate(dataloader_train):
        # zero the gradients
        decoder.zero_grad()
        encoder.zero_grad()

        # set decoder and encoder into train mode
        encoder.train()
        decoder.train()

        # Obtain the batch.
        images = data["image"]
        captions = data["caption"]
        # make the captions for targets and teacher forcer
        captions_target = captions[:, 1:].long().to(device)
        captions_train = captions[:, :captions.shape[1]-1].long().to(device)

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)

        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions_train)

        # Calculate the batch loss
        loss = criterion(outputs.view(-1, vocab_size), captions_target.contiguous().view(-1))

        # Backward pass
        loss.backward()

        # Update the parameters in the optimizer
        encoder_optimizer.step()
        decoder_optimizer.step()

       

        # - - - Validate - - -
        # turn the evaluation mode on
        if i % 100 == 0:
            with torch.no_grad():

                # set the evaluation mode
                encoder.eval()
                decoder.eval()

                # get the validation images and captions
                dataVal = next(iter(dataloader_val))
                val_images = dataVal["image"]
                img = val_images[0]
                val_captions = dataVal["caption"]
                cap = val_captions[0]
                
                caption = cap.cpu().detach().numpy()
                captionStr = [wordDict[i] for i in caption]
                print(captionStr)
                # define the captions
                captions_target = val_captions[:, 1:].long().to(device)
                captions_train = val_captions[:, :-1].long().to(device)

                # Move batch of images and captions to GPU if CUDA is available.
                val_images = val_images.to(device)

                # Pass the inputs through the CNN-RNN model.
                val_features = encoder(val_images)
                val_outputs = decoder(val_features, captions_train)
                
                captionOut = val_outputs[0].cpu().detach().numpy().argmax(axis = 1)
                captionOutStr = [wordDict[i] for i in captionOut]
                print("\n", captionOutStr)
                # Calculate the batch loss.
                val_loss = criterion(val_outputs.view(-1, vocab_size), captions_target.contiguous().view(-1))
                print("Loss of ", i, "th batch: ", val_loss.item(), sep="")

        # append the validation loss and training loss
        val_losses.append(val_loss.item())
#        losses.append(loss.item())
#
#        # save the losses
#        np.save("losses", np.array(losses))
#        np.save("val_losses", np.array(val_losses))
#
#        # Get training statistics.
#        stats = "Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Val Loss: %.4f" % (epoch, num_epochs, i_step, total_step, loss.item(), val_loss.item())
#
#        # Print training statistics (on same line).
#        print("\r" + stats, end="")
#        sys.stdout.flush()
#
#    # Save the weights.
#    if epoch % save_every == 0:
#        print("\nSaving the model")
#        torch.save(decoder.state_dict(), os.path.join("./models", "decoder-%d.pth" % epoch))
#        torch.save(encoder.state_dict(), os.path.join("./models", "encoder-%d.pth" % epoch))

cuda


/usr/local/lib/python3.6/dist-packages/tables/path.py:118: NaturalNameWarning: object name is a Python keyword: 'from'; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  % (name, _warnInfo), NaturalNameWarning)
/usr/local/lib/python3.6/dist-packages/tables/path.py:118: NaturalNameWarning: object name is a Python keyword: 'and'; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  % (name, _warnInfo), NaturalNameWarning)
/usr/local/lib/python3.6/dist-packages/tables/path.py:118: NaturalNameWarning: object name is a Python keyword: 'is'; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  % (name, _warnInfo), NaturalNameWarning)
/usr/local/lib/python3.6/dist-packages/tables/path.py:118: NaturalNameWarning: object name is a Python keyword: 'in'; you will not be able to use natural naming to access this object; 

Epoch: 1
['x_START_', 'a', 'kitchen', 'has', 'a', 'x_UNK_', 'cart', 'in', 'it', 'x_END_', 'x_NULL_', 'x_NULL_', 'x_NULL_', 'x_NULL_', 'x_NULL_', 'x_NULL_', 'x_NULL_']

 ['bus', 'couple', 'scooter', 'tower', 'couple', 'into', 'umpire', 'the', 'large', 'x_NULL_', 'x_NULL_', 'x_NULL_', 'x_NULL_', 'x_NULL_', 'x_NULL_', 'x_NULL_']
Loss of 0th batch: 6.4116411209106445


KeyboardInterrupt: ignored